# 動的なデフォルト引数を指定するときには None と docstring を扱う

In [ ]:
from time import sleep
from datetime import datetime

# 悪い例
def log(message, when=datetime.now()):
  print(f'{when}: {message}')

In [4]:
log('Hi there')
sleep(0.1) # sleep を入れているのに時刻が変わらない
log('Hi again')

2025-04-26 20:55:33.576404: Hi there
2025-04-26 20:55:33.576404: Hi again


タイムスタンプが同じなのは、datetime.now が関数が定義されたときの1回だけしか評価されないため

In [ ]:
# 良い例
# 期待した結果を得るためには、デフォルト値を None にして、
# docstring に実際の振る舞いを記述すること
def log(message, when=None):
  """Log a message with a timestamp.

  Args:
      message: Message to print
      when: datetime of when the message occurred.
          Defaults to the present time.

  """
  if when is None:
    when = datetime.now()
  print(f'{when}: {message}')

In [ ]:
print(log.__doc__)

Log a message with a timestamp.

  Args:
      message: Message to print
      when: datetime of when the message occurred.
          Defaults to the present time.

  


In [19]:
help(log)

Help on function log in module __main__:

log(message, when=None)
    Log a message with a timestamp.
    
    Args:
        message: Message to print
        when: datetime of when the message occurred.
            Defaults to the present time.



In [6]:
log('Hi there')
sleep(0.1)
log('Hi again')

2025-04-26 21:00:08.701394: Hi there
2025-04-26 21:00:08.801973: Hi again


In [12]:
import json

# 悪い例
# default に指定された辞書は1回しか評価されない
# ので、すべての decode の呼び出しで default が共有される
def decode(data, default={}):
  try:
    return json.loads(data)
  except ValueError:
    return default

In [11]:
foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1
# 2つの異なる辞書が、それぞれ1つのキーと値を持つことを期待している。
# が、実際は default が共有されるため、同じオブジェクトが出力される。
print('Foo:', foo)
print('Bar:', bar)

Foo: {'stuff': 5, 'meep': 1}
Bar: {'stuff': 5, 'meep': 1}


In [13]:
import json

# 良い例
def decode(data, default=None):
  """Load JSON data from a string.

  Args:
      data: JSON data to decode.
      default: Value to return if decoding fails.
          Defaults to an empty dictionary.

  """
  try:
    return json.loads(data)
  except ValueError:
    if default is None:
      default = {}
    return default

In [14]:
foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1
# 2つの異なる辞書が、それぞれ1つのキーと値を持つ。
print('Foo:', foo)
print('Bar:', bar)

Foo: {'stuff': 5}
Bar: {'meep': 1}


In [ ]:
# 以下の型ヒントでも動作する
from typing import Optional

# 引数の型ヒント name: type
# 戻り値の型ヒント -> type
# Optional は、指定の型 or None を許容する型
def log_typed(message: str, when: Optional[datetime]=None) -> None: # 関数の型ヒント
  """Log a message with a timestamp.

  Args:
      message: Message to print
      when: datetime of when the message occurred.
          Defaults to the present time.

  """
  if when is None:
    when = datetime.now()
  print(f'{when}: {message}')

In [17]:
log('Hi there')

2025-04-26 21:34:21.379142: Hi there
a: Hi there
